<a href="https://colab.research.google.com/github/dhoumaima/Information-Retrival/blob/main/ScrapyWebsite2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.8/287.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 9.8 MB/s eta 0:00:00


In [ ]:
!scrapy startproject MytimesNews

New Scrapy project 'MytimesNews', using template directory '/usr/local/lib/python3.10/dist-packages/scrapy/templates/project', created in:
    /content/MytimesNews

You can start your first spider with:
    cd MytimesNews
    scrapy genspider example example.com


In [ ]:
%cd MytimesNews/MytimesNews/spiders/

/content/MytimesNews/MytimesNews/spiders


In [ ]:
!scrapy genspider secondscraper https://www.nytimes.com/international/

Created spider 'secondscraper' using template 'basic' in module:
  MytimesNews.spiders.secondscraper


In [ ]:
%%writefile ./secondscraper.py
import scrapy


class secondscraper(scrapy.Spider):
    name = "secondscraper"
    allowed_domains = ["www.nytimes.com"]
    start_urls = ["https://www.nytimes.com/international"]
    news_count = 0

    def parse(self, response):
        news_items = response.css('.news')
        base_url = "https://www.nytimes.com"

        for news_item in news_items:
            news_url = f"{base_url}" + news_item.css('.title a::attr(href)').get()
            news_title = news_item.css('.title a::text').get()
            news_date = news_item.css('.infos::text').get()

            news_details = {
                'news_title': news_title,
                'news_date': news_date,
                'news_url': news_url
            }

            yield scrapy.Request(url=news_url, callback=self.parse_news_body, meta={'news_details': news_details})
            self.news_count += 1
            if self.news_count >= 150:
                break

        next_page = "https://www.nytimes.com/international"
        next_page_url = f"{next_page}" + response.css('.pagination a:contains("Suivant")::attr(href)').get()

        if next_page_url is not None and self.news_count < 150:
            yield scrapy.Request(next_page_url, callback=self.parse)

    def parse_news_body(self, response):
        news_content = response.css('.article_body p::text').extract()
        news_details = response.meta['news_details']
        news_details['content'] = news_content

        yield news_details

Overwriting ./secondscraper.py


In [ ]:
!scrapy crawl secondscraper -o news_data.json

2024-02-19 22:27:21 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: MytimesNews)
2024-02-19 22:27:21 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 24.0.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.2, Platform Linux-6.1.58+-x86_64-with-glibc2.35
2024-02-19 22:27:21 [scrapy.addons] INFO: Enabled addons:
[]
2024-02-19 22:27:21 [asyncio] DEBUG: Using selector: EpollSelector
2024-02-19 22:27:21 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2024-02-19 22:27:21 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2024-02-19 22:27:21 [scrapy.extensions.telnet] INFO: Telnet Password: fda0a79daed7dac6
2024-02-19 22:27:21 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',


In [ ]:
import json

with open('news_data.json','r') as file:
  data=json.load(file)

print(len(data))

0


je ne sais pas pourquoi data est vide
j'ai essayé de suivre des autres travaille mais ça ne donne pas la même resultat.